In [ ]:
import os
import numpy as np
import pandas as pd
import datetime as dt
from scipy import stats
import sqlalchemy as db
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *

In [ ]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

In [ ]:
query = 'SELECT * FROM BondReturns'

df = pd.read_sql(query, connection)

In [ ]:
df.head()

In [ ]:
df['R'] = df['R'].fillna(0)

In [ ]:
df.head()

In [ ]:
df['logR'] = np.log(1 + df['R'])

In [ ]:
df.head()

In [ ]:
J = 6 # formation period

df_ = df[
    ['CusipId', 'LtTrdExctnDt', 'R', 'logR']
].sort_values(['CusipId', 'LtTrdExctnDt']).set_index('LtTrdExctnDt')

cumr = df_.groupby(['CusipId'])['logR'].rolling(J).sum().reset_index()
cumr = cumr.rename(columns={'logR': 'SumLogR'})
cumr['CumRet']=np.exp(cumr['SumLogR'])-1

In [ ]:
cumr.tail(10)

In [ ]:
r = 10 # momentum ranking

cumr = cumr.dropna(axis=0, subset=['CumRet'])

cumr['MomR'] = cumr.groupby('LtTrdExctnDt')['CumRet'].transform(lambda x: pd.qcut(x, r, labels=False))
cumr['MomR'] = 1 + cumr['MomR'].astype(int)

In [ ]:
cumr.head()

In [ ]:
cumr.groupby('MomR')['CumRet'].mean()

In [ ]:
K = 6 # Holding Period Length: K can be between 3 to 12 months

cumr['form_date'] = cumr['LtTrdExctnDt']
cumr['medate'] = cumr['LtTrdExctnDt'] + MonthEnd(0)
cumr['hdate1'] = cumr['medate'] + MonthBegin(1)
cumr['hdate2'] = cumr['medate'] + MonthEnd(K)
cumr = cumr[['CusipId', 'form_date','MomR','hdate1','hdate2']]

In [ ]:
cumr.head()

In [ ]:
port = pd.merge(df[['CusipId', 'LtTrdExctnDt', 'R']], cumr, on=['CusipId'], how='inner')
port = port[(port['hdate1']<=port['LtTrdExctnDt']) & (port['LtTrdExctnDt']<=port['hdate2'])]
port = port[['CusipId','form_date', 'MomR', 'hdate1','hdate2', 'LtTrdExctnDt', 'R']]

In [ ]:
port.head()

In [ ]:
port['LtTrdExctnDt'] = pd.to_datetime(port['LtTrdExctnDt'])

In [ ]:
port.head()

In [ ]:
cumr_port = port.groupby(['LtTrdExctnDt','MomR', 'form_date'])['R'].mean().reset_index()
start_yr = cumr_port['LtTrdExctnDt'].dt.year.min()+2
cumr_port = cumr_port.loc[cumr_port.LtTrdExctnDt.dt.year>=start_yr]
cumr_port = cumr_port.sort_values(by=['LtTrdExctnDt','MomR'])

In [ ]:
cumr_port.head()

In [ ]:
ewret = cumr_port.groupby(['LtTrdExctnDt','MomR'])['R'].mean().reset_index()
ewstd = cumr_port.groupby(['LtTrdExctnDt','MomR'])['R'].std().reset_index()

ewret = ewret.rename(columns={'R_new':'ewret'})
ewstd = ewstd.rename(columns={'R_new':'ewretstd'})

ewretdf = pd.merge(ewret, ewstd, on=['LtTrdExctnDt','MomR'], how='inner')
ewretdf = ewretdf.sort_values(by=['MomR', 'LtTrdExctnDt'])

In [ ]:
ewretdf.head()

In [ ]:
ewretdf.groupby(['MomR'])['ewret'].describe()[['count','mean', 'std']].reset_index()

In [ ]:
ewret_t = ewretdf.pivot(index='LtTrdExctnDt', columns='MomR', values='ewret') 

# Add prefix port in front of each column 
ewret_t = ewret_t.add_prefix('port') 
ewret_t = ewret_t.rename(columns={'port1':'losers', 'port10':'winners'}) 
ewret_t['long_short'] = ewret_t.winners - ewret_t.losers

In [ ]:
ewret_t.head()

In [ ]:
ewret_t['cumret_winners'] = (1 + ewret_t.winners).cumprod()-1 
ewret_t['cumret_losers'] = (1 + ewret_t.losers).cumprod()-1
ewret_t['cumret_long_short'] = (1 + ewret_t.long_short).cumprod()-1

In [ ]:
mom_mean = ewret_t[['winners', 'losers', 'long_short']].mean().to_frame() 
mom_mean = mom_mean.rename(columns={0:'mean'}).reset_index() 
mom_mean

In [ ]:
t_losers = pd.Series(stats.ttest_1samp(ewret_t['losers'],0.0)).to_frame().T 
t_winners = pd.Series(stats.ttest_1samp(ewret_t['winners'],0.0)).to_frame().T 
t_long_short = pd.Series(stats.ttest_1samp(ewret_t['long_short'],0.0)).to_frame().T

t_losers['MomR']='losers' 
t_winners['MomR']='winners' 
t_long_short['MomR']='long_short'

In [ ]:
t_output =pd.concat([t_winners, t_losers, t_long_short]).rename(columns={0:'t-stat', 1:'p-value'}) 

# Combine mean, t and p and format output 
mom_output = pd.merge(mom_mean, t_output, on=['MomR'], how='inner')

mom_output['mean'] = mom_output['mean'].map('{:.2%}'.format) 
mom_output['t-stat'] = mom_output['t-stat'].map('{:.2f}'.format) 
mom_output['p-value'] = mom_output['p-value'].map('{:.2f}'.format) 

print('Momentum Strategy Summary:\n\n', mom_output)

In [ ]:
plt.figure(figsize=(12,9)) 
plt.suptitle('Momentum Strategy', fontsize=20) 

ax1 = plt.subplot(211) 
ax1.set_title('Long/Short Momentum Strategy', fontsize=15) 
ax1.plot(ewret_t['cumret_long_short']) 

ax2 = plt.subplot(212) 
ax2.set_title('Cumulative Momentum Portfolios', fontsize=15) 
ax2.plot(ewret_t['cumret_winners'], 'b-', ewret_t['cumret_losers'], 'r--') 
ax2.legend(('Winners','Losers'), loc='upper left', shadow=True) 

plt.subplots_adjust(top=0.92, hspace=0.2)